In [2]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [3]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser, # A synchronous browser is available, though it isn't compatible with jupyter.
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [4]:
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()

In [5]:
from langchain import hub

prompt = hub.pull("hwchase17/react-multi-input-json")

In [6]:
from langchain.tools.render import render_text_description_and_args

In [7]:
prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [8]:
prompt

ChatPromptTemplate(input_variables=['input', 'agent_scratchpad'], partial_variables={'tools': 'click_element: Click on an element with the given CSS selector, args: {\'selector\': {\'title\': \'Selector\', \'description\': \'CSS selector for the element to click\', \'type\': \'string\'}}\nnavigate_browser: Navigate a browser to the specified URL, args: {\'url\': {\'title\': \'Url\', \'description\': \'url to navigate to\', \'type\': \'string\'}}\nprevious_webpage: Navigate back to the previous page in the browser history, args: {}\nextract_text: Extract all the text on the current webpage, args: {}\nextract_hyperlinks: Extract all hyperlinks on the current webpage, args: {\'absolute_urls\': {\'title\': \'Absolute Urls\', \'description\': \'Return absolute URLs instead of relative URLs\', \'default\': False, \'type\': \'boolean\'}}\nget_elements: Retrieve elements in the current web page matching the given CSS selector, args: {\'selector\': {\'title\': \'Selector\', \'description\': "CS

In [9]:
from configs.config import configs
from langchain import OpenAI 
OpenAI.api_key = configs.get("OPENAI_API_KEY")

llm = ChatOpenAI(temperature=0)
llm_with_stop = llm.bind(stop=["Observation"])

In [10]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

In [11]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
} | prompt | llm_with_stop | JSONAgentOutputParser()

In [12]:
from langchain.agents import AgentExecutor

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
response = await agent_executor.ainvoke({"input": "Browse to blog.langchain.dev and summarize the text, please."})
print(response['output'])



> Entering new AgentExecutor chain...


Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```
Navigating to https://blog.langchain.dev returned status code 200Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```

LangChain Blog Skip to content LangChain Blog Home By LangChain Release Notes GitHub Docs Write with Us Sign in Subscribe Using a Knowledge Graph to implement a DevOps RAG application Editor's Note: This post was written in collaboration with the Neo4J team.

RAG applications are all the rage at the moment. Everyone is building their 8 min read Featured Building Chat LangChain 7 min read Announcing LangChain Hub By LangChain 6 min read Using LangSmith to Support Fine-tuning By LangChain 9 min read Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications 8 min read Building (and Breaking) WebLangChain Important Links:

 * Hosted WebLangChain
 * Open-source code for WebLangChain


Intr

In [15]:
llm = ChatOpenAI(temperature=0) # Also works well with Anthropic models
agent_chain = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [18]:
response = await agent_chain.ainvoke({"input": "Browse to https://tecelit.hashnode.dev/iot-use-case-with-flask-and-http-part-2 and summarize the text, please."})
print(response['output'])



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://tecelit.hashnode.dev/iot-use-case-with-flask-and-http-part-2"
  }
}
```

Observation: Navigating to https://tecelit.hashnode.dev/iot-use-case-with-flask-and-http-part-2 returned status code 200
Thought:The webpage has been successfully loaded. Now I need to extract the text from the webpage to summarize it.
Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```
Observation: IOT use-case with Flask and HTTP (Part 2) Follow Follow IOT use-case with Flask and HTTP (Part 2) TecElit Blog · May 1, 2021 · 7 min read In the previous tutorial we learnt about how HTTP can be used in IOT ie. for use-cases where a sensor needs to send data to the server at regular intervals. We learnt about a backend framework - Flask and how it helps us serving the API endpoints for GET and POST requests on the server. Then we discussed about a real-world problem, that occurs